In [14]:
import sqlite3
from faker import Faker
from random import randint
import random

In [15]:
def generate_cpf():
    # Generate the first 9 digits of the CPF randomly
    cpf = [random.randint(0, 9) for _ in range(9)]
    
    # Calculate the first verification digit
    sum1 = sum((10 - i) * cpf[i] for i in range(9))
    d1 = 11 - (sum1 % 11)
    if d1 >= 10:
        d1 = 0
    cpf.append(d1)
    
    # Calculate the second verification digit
    sum2 = sum((11 - i) * cpf[i] for i in range(10))
    d2 = 11 - (sum2 % 11)
    if d2 >= 10:
        d2 = 0
    cpf.append(d2)
    
    # Format the CPF as XXX.XXX.XXX-YY
    cpf_str = f"{''.join(map(str, cpf[:3]))}.{''.join(map(str, cpf[3:6]))}.{''.join(map(str, cpf[6:9]))}-{''.join(map(str, cpf[9:]))}"
    return cpf_str

# Example usage
print([generate_cpf() for _ in range(20)])


['544.732.010-07', '688.698.174-90', '664.284.192-11', '352.605.671-45', '636.664.917-07', '198.793.399-08', '263.403.921-21', '198.819.166-15', '465.605.988-96', '240.640.334-34', '775.967.907-63', '014.256.681-00', '074.061.178-09', '855.149.682-43', '057.736.139-21', '826.575.339-06', '704.364.080-66', '632.335.743-71', '000.667.140-36', '886.839.496-05']


In [20]:
# Initialize Faker
faker = Faker()

# Connect to SQLite
conn = sqlite3.connect('fake.sqlite3')
cursor = conn.cursor()

In [21]:
N = 100

for _ in range(N):
    user_id = faker.random_int(min=1000, max=100000)
    phone_number = faker.phone_number()
    address = faker.address().replace('\n', ', ')
    username = faker.user_name()
    cpf = faker.random_element([generate_cpf() for _ in range(N)])
    is_psychologist = random.choice([True, False])
    cursor.execute('''
        INSERT INTO appsico_app_patients (user_ptr_id, phonenumber, address, user_name, cpf, is_psicologist)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (user_id, phone_number, address, username, cpf, is_psychologist))

conn.commit()
conn.close()

In [ ]:
# Get all CPFs for use in transactions
cursor.execute('SELECT cpf_user_id FROM appsico_app_calendar')

# Insert fake data into transactions
range_ = range(20)
for _ in range_:
    duration = randint(1000, 10000)  # Random duration in seconds
    date = faker.date_between(start_date='-1y', end_date='today')  # Random date within the last year
    cpf = faker.random_element([generate_cpf() for _ in range_])  # Pick a random CPF from the persons table
    cursor.execute('INSERT INTO appsico_app_calendar (duration, data, cpf_user_id) VALUES (?, ?, ?)', (duration, date, cpf))
    
    
conn.commit()
conn.close()
